In [ ]:
using Revise

In [ ]:
using RigidBodyDynamics
using LearningMPC
using Gurobi
using DrakeVisualizer
using CoordinateTransformations
using Plots; gr()
using ProgressMeter
using MLDataPattern
using JLD2
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()

In [ ]:
import BoxValkyries
import Nets

In [ ]:
reload("BoxValkyries")

In [ ]:
const Sample = Tuple{Vector{Float64}, Matrix{Float64}}

In [ ]:
boxval = BoxValkyries.BoxValkyrie()
mechanism = boxval.mechanism
basevis = Visualizer()[:boxval]
delete!(basevis)
setgeometry!(basevis, boxval)


nx = num_positions(mechanism) + num_velocities(mechanism)
nu = num_velocities(mechanism)

activation = Nets.leaky_relu
function sensitive_loss(λ)
    q = fill(λ, 1, 1 + nx)
    q[1] = 1 - λ
    (params, x, y) -> sum(abs2, 
        q .* (Nets.predict_sensitivity(Nets.Net(Nets.Params(widths, params), activation, x_to_u, v_to_y), x) .- y))
end

p_train = 0.6
p_validate = 0.2

In [ ]:
mpc_params = BoxValkyries.BoxValkyrieMPCParams()
xstar = MechanismState{Float64}(mechanism)
set_configuration!(xstar, findjoint(mechanism, "base_x"), [0.4])
set_configuration!(xstar, findjoint(mechanism, "base_z"), [0.9])
set_configuration!(xstar, findjoint(mechanism, "core_to_rh_x"), [0.6])
set_configuration!(xstar, findjoint(mechanism, "core_to_rh_z"), [0.0])
set_configuration!(xstar, findjoint(mechanism, "core_to_lh_x"), [0.6])
set_configuration!(xstar, findjoint(mechanism, "core_to_lh_z"), [0.0])
set_configuration!(xstar, findjoint(mechanism, "core_to_rf_x"), [0.2])
set_configuration!(xstar, findjoint(mechanism, "core_to_rf_z"), [-0.9])
set_configuration!(xstar, findjoint(mechanism, "core_to_lf_x"), [0.2])
set_configuration!(xstar, findjoint(mechanism, "core_to_lf_z"), [-0.9])
settransform!(basevis[:robot], xstar)

feet = findbody.(mechanism, ["rf", "lf"])
contacts = [Point3D(default_frame(body), 0., 0, 0) for body in feet]
lqrsol = BoxValkyries.LQRSolution(xstar, BoxValkyries.nominal_input(boxval, xstar), mpc_params.Q, mpc_params.R, contacts)
lqr_controller = BoxValkyries.LQRController(lqrsol)

In [ ]:
x0 = MechanismState(mechanism, copy(configuration(xstar)), copy(velocity(xstar)))
set_velocity!(x0, findjoint(mechanism, "base_x"), [-0.2])
set_configuration!(x0, findjoint(mechanism, "base_z"), [1.2])
q0 = copy(configuration(x0))
v0 = copy(velocity(x0));

In [ ]:
# set_configuration!(x0, q0)
# set_velocity!(x0, v0)
# env = Gurobi.Env()
# results_lqr = LCPSim.simulate(x0, lqr_controller, boxval.environment, mpc_params.Δt, 200, GurobiSolver(env, OutputFlag=0));

# playback(basevis[:robot], results_lqr, mpc_params.Δt)

In [ ]:
param_file = "box-val-dagger-0.04-v2-params.jld2"
data_file = "box-val-dagger-0.04-v2.jld2"

In [ ]:
# params, widths, x_to_u, v_to_y = jldopen(param_file, "r") do file
#     params = file["params"]
#     widths = file["widths"]
#     x_to_u = file["x_to_u"]
#     v_to_y = file["v_to_y"]
#     params, widths, x_to_u, v_to_y
# end;

# train_data, test_data, validation_data = jldopen(data_file, "r") do file
#     train_data = file["train_data"]
#     test_data = file["test_data"]
#     validation_data = file["validation_data"]
#     train_data, test_data, validation_data
# end;

In [ ]:
start_params = 0.1 * randn(Nets.Params{Float64}, widths).data;
params = copy(start_params)
x_to_u = AffineMap(eye(nx), zeros(nx))
v_to_y = AffineMap(diagm([max(abs(b.lower), abs(b.upper)) for b in LCPSim.all_effort_bounds(mechanism)]), zeros(num_velocities(mechanism)))

widths = [nx, 16, 16, 16, 16, nu]

train_data = Sample[]
test_data = Sample[]
validation_data = Sample[]

In [ ]:
net = Nets.Net(Nets.Params(widths, params), activation, x_to_u, v_to_y)

In [ ]:
net_controller = x -> begin
    Nets.predict(net, state_vector(x))
end

last_trajectory = []

mpc_controller = BoxValkyries.MPCController(boxval, mpc_params, lqrsol, [net_controller, lqr_controller]);
mpc_controller.callback = (x, results) -> begin
    if !isnull(results.lcp_updates)
        global last_trajectory
        last_trajectory = get(results.lcp_updates)
        playback(basevis[:robot], get(results.lcp_updates), mpc_controller.params.Δt)
        if !isnull(results.jacobian)
            xv = state_vector(x)
            yJ = hcat(get(results.lcp_updates)[1].input, get(results.jacobian))
            r = rand()
            if r < p_train
                push!(train_data, (xv, yJ))
            elseif r < p_train + p_validate
                push!(validation_data, (xv, yJ))
            else
                push!(test_data, (xv, yJ))
            end
        end
    end
end

train_loss = sensitive_loss(0.1)
validate_loss = train_loss
training_losses = Float64[]
validation_losses = Float64[]

p_mpc = 0.2

x_control = MechanismState{Float64}(mechanism)

controller = x -> begin
    set_configuration!(x_control, configuration(x))
    settransform!(basevis[:robot], x_control)
    if rand() < p_mpc
        return mpc_controller(x)
    else
        return net_controller(x)
    end
end

In [ ]:
x0 = MechanismState{Float64}(mechanism)
q0 = configuration(xstar) .+ 0.0 .* randn(num_positions(xstar))
v0 = copy(velocity(xstar)) .+ 0.0 * randn(num_velocities(xstar))
v0[1] = 1.5
set_configuration!(x0, q0)
set_velocity!(x0, v0)
mpc_controller(x0)


In [ ]:
playback(basevis[:robot], last_trajectory, mpc_params.Δt)

In [ ]:
length(params)

In [ ]:
x0 = MechanismState{Float64}(mechanism)
q0 = configuration(xstar) .+ 0.0 .* randn(num_positions(xstar))
v0 = copy(velocity(xstar)) .+ 0.0 * randn(num_velocities(xstar))
v0[1] = 1.5
set_configuration!(x0, q0)
set_velocity!(x0, v0)
solver = GurobiSolver(Gurobi.Env(silent=true), OutputFlag=0)
results = LCPSim.simulate(x0, mpc_controller, boxval.environment, mpc_params.Δt, 50, solver);

In [ ]:
playback(basevis[:robot], results, 0.05)

In [ ]:
x0 = MechanismState{Float64}(mechanism)

env = Gurobi.Env()

@showprogress for i in 1:1000
    q = copy(configuration(xstar))
    q[1] += 0.25 * randn()
    set_configuration!(x0, q)
    set_velocity!(x0, velocity(xstar) .+ randn(size(v0)))
    
#     set_configuration!(x0, [2 * (rand() - 0.5), π * (rand() - 0.5)])
#     set_velocity!(x0, (rand(2) .- 0.5))
    termination = x -> (configuration(x)[2] < 0.2)
    results_net = LCPSim.simulate(x0, controller, boxval.environment, mpc_params.Δt, 100, GurobiSolver(env, OutputFlag=0);
        termination=termination)
    Nets.adam!(train_loss, params, shuffleobs(train_data), Nets.AdamOpts(learning_rate=0.01 * 0.999^i, batch_size=min(10, length(train_data))))
    push!(training_losses, mean(xy -> validate_loss(params, xy[1], xy[2]), train_data))
    push!(validation_losses, mean(xy -> validate_loss(params, xy[1], xy[2]), validation_data))
    
    @show training_losses[end]
    @show validation_losses[end]
    
    jldopen(data_file, "w") do file
        file["train_data"] = train_data
        file["test_data"] = test_data
        file["validation_data"] = validation_data
    end
    
    jldopen(param_file, "w") do file
        file["params"] = params
        file["widths"] = widths
        file["x_to_u"] = x_to_u
        file["v_to_y"] = v_to_y
        file["training_losses"] = training_losses
        file["validation_losses"] = validation_losses
    end;
end 

In [ ]:
q0 = configuration(xstar) .+ 0.1 .* randn(num_positions(xstar))
v0 = copy(velocity(xstar));

In [ ]:
set_configuration!(x0, q0)
set_velocity!(x0, v0)
results_net = LCPSim.simulate(x0, net_controller, boxval.environment, mpc_params.Δt, 300, GurobiSolver(Gurobi.Env(), OutputFlag=0));

In [ ]:
set_configuration!(x0, configuration(results_net[1].state))
settransform!(basevis[:robot], x0)

In [ ]:
set_configuration!(x0, configuration(results_net[1].state))
settransform!(basevis[:robot], x0)

playback(basevis[:robot], results_net, 0.5 * mpc_params.Δt)

In [ ]:
set_configuration!(x0, q0)
set_velocity!(x0, v0)
results_lqr = LCPSim.simulate(x0, lqr_controller, boxval.environment, 
    mpc_params.Δt, 300, GurobiSolver(Gurobi.Env(), OutputFlag=0));

In [ ]:
set_configuration!(x0, configuration(results_lqr[1].state))
settransform!(basevis[:robot], x0)

playback(basevis[:robot], results_lqr, mpc_params.Δt)

In [ ]:
online_controller = BoxValkyries.OnlineMPCController(boxval, 
    mpc_params, lqrsol, [net_controller, lqr_controller]);

In [ ]:
set_configuration!(x0, q0)
set_velocity!(x0, v0)
results_online = LCPSim.simulate(x0, online_controller, boxval.environment, mpc_params.Δt, 100, GurobiSolver(Gurobi.Env(), OutputFlag=0));

In [ ]:
playback(basevis[:robot], results_online, mpc_params.Δt)

In [ ]:
using ProfileView
set_configuration!(x0, q0)
set_velocity!(x0, v0)
online_controller(x0)
Profile.clear()
@time @profile online_controller(x0)
ProfileView.view()